## Камера

In [1]:
from jetcam.csi_camera import CSICamera
# from jetcam.usb_camera import USBCamera

camera = CSICamera(width=224, height=224)
# camera = USBCamera(width=224, height=224)

camera.running = True

RuntimeError: Could not initialize camera.  Please see error trace.

## Задача

In [2]:
import torchvision.transforms as transforms
from xy_dataset import XYDataset

TASK = 'road_following'

CATEGORIES = ['apex']

DATASETS = ['A', 'B', "automatic", "automatic_mini", "automatic_observer", "automatic_loop"]

TRANSFORMS = transforms.Compose([
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

datasets = {}
for name in DATASETS:
    datasets[name] = XYDataset(TASK + '_' + name, CATEGORIES, TRANSFORMS, random_hflip=True)

## Controller

In [3]:
import ipywidgets.widgets as widgets

controller = widgets.Controller(index=0)  # replace with index of your controller

# display(controller)
print("Controller is made!")

Controller is made!


## Teleoperations

In [4]:
from jetracer.nvidia_racecar import NvidiaRacecar
import traitlets

car = NvidiaRacecar()

car.throttle_gain = 0.2

car.steering_offset=0.1
car.steering = 0

left_link = traitlets.dlink((controller.axes[0], 'value'), (car, 'steering'), transform=lambda x: -x)
right_link = traitlets.dlink((controller.axes[3], 'value'), (car, 'throttle'), transform=lambda x: x)

## виджеты данных

In [5]:
import ipywidgets.widgets as widgets
import cv2
import ipywidgets
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
from jupyter_clickable_image_widget import ClickableImageWidget


# initialize active dataset
dataset = datasets[DATASETS[0]]

# unobserve all callbacks from camera in case we are running this cell for second time
camera.unobserve_all()

# create image preview
camera_widget = ClickableImageWidget(width=camera.width, height=camera.height)
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

# create widgets
dataset_widget = ipywidgets.Dropdown(options=DATASETS, description='dataset')
category_widget = ipywidgets.Dropdown(options=dataset.categories, description='category')
count_widget = ipywidgets.IntText(description='count')
state_widget = ipywidgets.ToggleButtons(options=['stop', 'start'], description='start record data operator', value='stop')

# manually update counts at initialization
count_widget.value = dataset.get_count(category_widget.value)

# sets the active dataset
def set_dataset(change):
    global dataset
    dataset = datasets[change['new']]
    count_widget.value = dataset.get_count(category_widget.value)
dataset_widget.observe(set_dataset, names='value')

# update counts when we select a new category
def update_counts(change):
    count_widget.value = dataset.get_count(change['new'])
category_widget.observe(update_counts, names='value')

# -------------------------------------------------------------------------------------------------------------------------- #

x_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='x (left joystick)')
y_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='y (right joystick)', orientation='vertical')
count_widget = ipywidgets.IntText(description='count')

# sets the active dataset
def set_dataset(change):
    global dataset
    dataset = datasets[change['new']]
    count_widget.value = dataset.get_count(category_widget.value)
dataset_widget.observe(set_dataset, names='value')

display(
    ipywidgets.VBox([
        ipywidgets.HBox([camera_widget, x_slider, y_slider]),
        dataset_widget,
        category_widget,
        count_widget,
        state_widget
    ])
)

# Создание БД для работы в Python с базой данных Sqlite для записи обучающей выборки

In [6]:
import sqlite3, datetime

connection = sqlite3.connect('train_database.db')

cursor = connection.cursor()

# Создаем таблицу TrainData для сохранения данных об обучающей выборки
cursor.execute('''
CREATE TABLE IF NOT EXISTS TrainData (
id INTEGER PRIMARY KEY,
task TEXT NOT NULL,
category TEXT NOT NULL,
dataset TEXT NOT NULL,
file_name INTEGER NOT NULL,
x TEXT NOT NULL,
y TEXT NOT NULL,
datetime TEXT NOT NULL
)
''')

# Сохраняем изменения и закрываем соединение
connection.commit()
connection.close()

# Автоматический сбор данных через паттер Наблюдатель "Observer"

In [7]:
import datetime
import sqlite3

# создаем объект работы с локальной БД Sqlite
connection = sqlite3.connect('train_database.db')
# устанавливаем соединение с локальной базой данных
cursor = connection.cursor()

def call_back_for_observer_axes(change):
    if change['new']:
        x = controller.axes[0].value
        y = controller.axes[3].value * -1 
        x_slider.value = x
        y_slider.value = y
        
        # save to disk
        filename = dataset.save_entry(category_widget.value, camera.value, x, y)
    
        # Добавляем новую строку данных об процессе обучения
        cursor.execute('INSERT INTO TrainData (task, category, dataset, file_name, x, y, datetime) VALUES (?, ?, ?, ?, ?, ?, ?)', 
            (
                TASK, # задача
                category_widget.value, # категория
                dataset_widget.value, # dataset
                filename, # наименование файла
                str(x), # значение координат по оси Х
                str(y), # значение координат по оси Y 
                datetime.datetime.now(), # точка времени
            )
        )
        
        # Сохраняем изменения в локальной БД Sqlite3
        connection.commit()
    
        count_widget.value = dataset.get_count(category_widget.value)
        
controller.axes[0].observe(call_back_for_observer_axes, names='value')
controller.axes[3].observe(call_back_for_observer_axes, names='value') 

# Автоматический сбор данных в цикле с выделенным потоком

In [7]:
import threading
import time
import datetime
import sqlite3


def loop_func_record_data(state_widget, controller, dataset, x_slider, y_slider, count_widget, task, category_widget, dataset_widget):
    # создаем объект работы с локальной БД Sqlite
    connection = sqlite3.connect('train_database.db')
    # устанавливаем соединение с локальной базой данных
    cursor = connection.cursor()
    while state_widget.value == "start":
        time.sleep(0.1) # задержка для меньшей чистоты кадров
        x = controller.axes[0].value
        y = controller.axes[3].value * -1 
        x_slider.value = x
        y_slider.value = y
        
        # save to disk   
        filename = dataset.save_entry(category_widget.value, camera.value, x, y)
        
        # сохранение в локльной БД Sqlite3
                       
        # Добавляем новую строку данных об процессе обучения
        cursor.execute('INSERT INTO TrainData (task, category, dataset, file_name, x, y, datetime) VALUES (?, ?, ?, ?, ?, ?, ?)', 
            (
                task, # задача
                category_widget.value, # категория
                dataset_widget.value, # dataset
                filename, # наименование файла
                str(x), # значение координат по оси Х
                str(y), # значение координат по оси Y 
                datetime.datetime.now(), # точка времени
            )
        )
        
        # Сохраняем изменения в локальной БД Sqlite3
        connection.commit()
        
        count_widget.value = dataset.get_count(category_widget.value)
    
def start_live(change):
    if change['new'] == 'start':
        execute_thread = threading.Thread(target=loop_func_record_data, 
                args=(state_widget, controller, dataset, x_slider, y_slider, count_widget, TASK, category_widget, dataset_widget))
        execute_thread.start()
    
state_widget.observe(start_live, names='value')